# Welcome to an Individual Home Run Spray Chart Application for the seasons 2018 - 2024! (read instructions below)

**1**: type in the name of the player that you want to analyze

**2**: type in the season that you want to analyze

**3**: Hit 'Run All' at the top
  
**4**: Scroll down and after a few seconds an animation will play showing all the player's home runs for that season as well as the date, pitcher, launch angle, exit velocity, pitch velocity, and pitch type.

*Notes:* You can hit 'Pause' to stop the animation or 'Skip to End' to bring
the animation to the end. Once you are at the end, you can click on different
homeruns on the slider at the bottom -- the brightest dot on the screen correlates to that home run. You can either look at the table for the relevant information or hover over said brightest dot.

Thanks and I hope you enjoy!

In [5]:
# @title Type in the name and year of the player (2018-2024) for home run spray
#             chart and then click 'run all' and scroll down

!pip -q install pybaseball plotly

PLAYER = "Mike Trout"  # @param {type:"string"}
YEAR   = 2024         # @param {type:"integer"}

import pandas as pd
pd.options.display.float_format = "{:,.2f}".format


In [6]:
# @title writefile mlb_vis_config.py
%%writefile mlb_vis_config.py

import numpy as np

# Regular-season date windows by year
REG_SEASON = {
    2018: ("2018-03-29", "2018-10-01"),
    2019: ("2019-03-20", "2019-09-29"),
    2020: ("2020-07-23", "2020-09-27"),
    2021: ("2021-04-01", "2021-10-03"),
    2022: ("2022-04-07", "2022-10-05"),
    2023: ("2023-03-30", "2023-10-01"),
    2024: ("2024-03-20", "2024-09-29"),
    2025: ("2025-03-18", "2025-09-28"),
}

# Outfield fence approximations (ft) by 5 - point designs (corners, alleys, center)
PARKS = {
    "BAL": dict(LF=333, LCF=384, CF=400, RCF=373, RF=318), "BOS": dict(LF=310, LCF=379, CF=390, RCF=380, RF=302),
    "NYY": dict(LF=318, LCF=399, CF=408, RCF=385, RF=314), "TB":  dict(LF=315, LCF=370, CF=404, RCF=370, RF=322),
    "TOR": dict(LF=328, LCF=375, CF=400, RCF=375, RF=328), "CWS": dict(LF=330, LCF=375, CF=400, RCF=375, RF=335),
    "CLE": dict(LF=325, LCF=370, CF=405, RCF=375, RF=325), "DET": dict(LF=342, LCF=370, CF=412, RCF=365, RF=342),
    "KC":  dict(LF=330, LCF=387, CF=410, RCF=387, RF=330), "MIN": dict(LF=339, LCF=377, CF=404, RCF=367, RF=328),
    "HOU": dict(LF=315, LCF=362, CF=409, RCF=373, RF=326), "LAA": dict(LF=347, LCF=387, CF=396, RCF=370, RF=350),
    "OAK": dict(LF=330, LCF=367, CF=400, RCF=367, RF=330), "SEA": dict(LF=331, LCF=378, CF=401, RCF=381, RF=326),
    "TEX": dict(LF=329, LCF=372, CF=407, RCF=374, RF=326), "ATL": dict(LF=335, LCF=375, CF=400, RCF=375, RF=325),
    "MIA": dict(LF=344, LCF=387, CF=407, RCF=382, RF=335), "NYM": dict(LF=335, LCF=370, CF=408, RCF=370, RF=330),
    "PHI": dict(LF=329, LCF=374, CF=401, RCF=369, RF=330), "WSH": dict(LF=336, LCF=377, CF=402, RCF=370, RF=335),
    "CHC": dict(LF=355, LCF=368, CF=400, RCF=368, RF=353), "CIN": dict(LF=328, LCF=379, CF=404, RCF=370, RF=325),
    "MIL": dict(LF=344, LCF=371, CF=400, RCF=374, RF=345), "PIT": dict(LF=325, LCF=389, CF=399, RCF=375, RF=320),
    "STL": dict(LF=336, LCF=385, CF=400, RCF=375, RF=335), "ARI": dict(LF=330, LCF=374, CF=407, RCF=374, RF=334),
    "COL": dict(LF=347, LCF=390, CF=415, RCF=375, RF=350), "LAD": dict(LF=330, LCF=375, CF=395, RCF=375, RF=330),
    "SDP": dict(LF=336, LCF=390, CF=396, RCF=391, RF=322), "SF":  dict(LF=339, LCF=364, CF=391, RCF=421, RF=309),
}

# ESPN logo URLs (PNG)
LOGO_URLS = {
    "ARI":"https://a.espncdn.com/i/teamlogos/mlb/500/ari.png","ATL":"https://a.espncdn.com/i/teamlogos/mlb/500/atl.png",
    "BAL":"https://a.espncdn.com/i/teamlogos/mlb/500/bal.png","BOS":"https://a.espncdn.com/i/teamlogos/mlb/500/bos.png",
    "CHC":"https://a.espncdn.com/i/teamlogos/mlb/500/chc.png","CIN":"https://a.espncdn.com/i/teamlogos/mlb/500/cin.png",
    "COL":"https://a.espncdn.com/i/teamlogos/mlb/500/col.png","LAD":"https://a.espncdn.com/i/teamlogos/mlb/500/lad.png",
    "SF":"https://a.espncdn.com/i/teamlogos/mlb/500/sf.png","SDP":"https://a.espncdn.com/i/teamlogos/mlb/500/sd.png",
    "SEA":"https://a.espncdn.com/i/teamlogos/mlb/500/sea.png","OAK":"https://a.espncdn.com/i/teamlogos/mlb/500/oak.png",
    "TEX":"https://a.espncdn.com/i/teamlogos/mlb/500/tex.png","TOR":"https://a.espncdn.com/i/teamlogos/mlb/500/tor.png",
    "CWS":"https://a.espncdn.com/i/teamlogos/mlb/500/chw.png","CLE":"https://a.espncdn.com/i/teamlogos/mlb/500/cle.png",
    "DET":"https://a.espncdn.com/i/teamlogos/mlb/500/det.png","KC":"https://a.espncdn.com/i/teamlogos/mlb/500/kc.png",
    "MIN":"https://a.espncdn.com/i/teamlogos/mlb/500/min.png","NYY":"https://a.espncdn.com/i/teamlogos/mlb/500/nyy.png",
    "NYM":"https://a.espncdn.com/i/teamlogos/mlb/500/nym.png","PHI":"https://a.espncdn.com/i/teamlogos/mlb/500/phi.png",
    "PIT":"https://a.espncdn.com/i/teamlogos/mlb/500/pit.png","STL":"https://a.espncdn.com/i/teamlogos/mlb/500/stl.png",
    "MIA":"https://a.espncdn.com/i/teamlogos/mlb/500/mia.png","MIL":"https://a.espncdn.com/i/teamlogos/mlb/500/mil.png",
    "WSH":"https://a.espncdn.com/i/teamlogos/mlb/500/wsh.png","TB":"https://a.espncdn.com/i/teamlogos/mlb/500/tb.png",
    "HOU":"https://a.espncdn.com/i/teamlogos/mlb/500/hou.png","LAA":"https://a.espncdn.com/i/teamlogos/mlb/500/laa.png"
}

# Primary Team Colors
TEAM_COLORS = {
    "ARI":"#A71930","ATL":"#CE1141","BAL":"#DF4601","BOS":"#BD3039","CHC":"#0E3386","CWS":"#27251F",
    "CIN":"#C6011F","CLE":"#E31937","COL":"#33006F","DET":"#0C2340","HOU":"#EB6E1F","KC":"#004687",
    "LAA":"#BA0021","LAD":"#005A9C","MIA":"#00A3E0","MIL":"#12284B","MIN":"#002B5C","NYM":"#FF5910",
    "NYY":"#0C2340","OAK":"#003831","PHI":"#E81828","PIT":"#FDB827","SDP":"#2F241D","SF":"#FD5A1E",
    "SEA":"#0C2C56","STL":"#C41E3A","TB":"#8FBCE6","TEX":"#003278","TOR":"#134A8E","WSH":"#AB0003"
}

def team_color(code: str) -> str:
    """Return the primary team color, backup is black."""
    k = str(code).upper() if code else ""
    return TEAM_COLORS.get(k, "#000000")

def logo_url(code: str) -> str:
    """Return the ESPN PNG logo URL for the logo in center field; backup is empty."""
    k = str(code).upper() if code else ""
    return LOGO_URLS.get(k, "")

def diamond_points(side: float = 90.0) -> np.ndarray:
    """Creates the infield diamond - square rotated 45 degrees."""
    d = side / np.sqrt(2.0)
    home  = (0.0, 0.0)
    first = (-d, d)
    second= (0.0, side*np.sqrt(2.0))
    third = (d, d)
    return np.array([home, first, second, third], dtype=float)

def park_outline(team_code: str, n: int = 361):
    """Outlines the outfield wall using the 5 - point outfield fence approximations."""
    import numpy as _np
    k = str(team_code).upper() if team_code else ""
    p = PARKS.get(k)
    if p is None:
        raise KeyError(f"Unknown team code '{k}'. Expected one of: {', '.join(sorted(PARKS))}")
    ang_deg = _np.array([-45, -20, 0, 20, 45], dtype=float)
    dist    = _np.array([p["LF"], p["LCF"], p["CF"], p["RCF"], p["RF"]], dtype=float)
    thetas  = _np.linspace(-45, 45, n)
    rs      = _np.interp(thetas, ang_deg, dist)
    rad     = _np.deg2rad(thetas)
    x = rs * _np.sin(rad)
    y = rs * _np.cos(rad)
    return x, y, dict(LF=p["LF"], LCF=p["LCF"], CF=p["CF"], RCF=p["RCF"], RF=p["RF"])


Overwriting mlb_vis_config.py


In [7]:
# @title writefile mlb_vis_core.py
%%writefile mlb_vis_core.py

import numpy as np
import pandas as pd
import plotly.graph_objects as go

from pybaseball import statcast_batter, playerid_lookup
try:
    from pybaseball import playerid_reverse_lookup
except Exception:
    playerid_reverse_lookup = None

from mlb_vis_config import (
    REG_SEASON, team_color, logo_url, diamond_points
)


def season_dates(year: int):
    """Return start and end date for year in 2018-2024."""
    if year not in REG_SEASON:
        raise ValueError(f"Season dates for {year} are not defined in REG_SEASON.")
    return REG_SEASON[year]


def lookup_mlbam_id(fullname: str) -> int:
    """Looks up a player's MLBAM id from full name string given above using pybaseball's 'player_idlookup."""
    parts = fullname.strip().split()
    last = parts[-1]
    first = " ".join(parts[:-1]) if len(parts) > 1 else ""
    ids = playerid_lookup(last, first)
    if ids.empty:
        raise ValueError(f"Player not found: {fullname}")
    ids = ids.sort_values(["mlb_played_last", "mlb_played_first"], ascending=[False, False])
    return int(ids.iloc[0]["key_mlbam"])


def statcast_xy_to_feet(hc_x, hc_y):
    """Convert Statcast spray coordinates in pixels to feet from home plate."""
    x = (hc_x - 125) * 2.2857
    y = (198 - hc_y) * 2.2857
    return x, y


def pick_primary_home_team(df: pd.DataFrame) -> str:
    """Choose the player's primary home team from event rows to to choose color and park construction."""
    bot = df[df["inning_topbot"].astype(str).str.upper() == "BOT"]
    if bot.empty:
        return str(df["home_team"].mode().iloc[0])
    return str(bot["home_team"].mode().iloc[0])


def ball_flight_path(x1: float, y1: float, steps: int = 30, curve: float = 0.05):
    """Generate a simple, smooth flight path from home plate to where ball lands using a Bezier curve I chose."""
    x0, y0 = 0.0, 0.0
    t = np.linspace(0.0, 1.0, steps)
    dx, dy = x1 - x0, y1 - y0
    mx, my = x0 + dx * 0.5, y0 + dy * 0.5
    nx, ny = -dy, dx
    n = np.hypot(nx, ny) or 1.0
    nx, ny = nx / n, ny / n
    lift = curve * np.hypot(dx, dy)
    cx, cy = mx + lift * nx, my + lift * ny
    px = (1 - t) ** 2 * x0 + 2 * (1 - t) * t * cx + t ** 2 * x1
    py = (1 - t) ** 2 * y0 + 2 * (1 - t) * t * cy + t ** 2 * y1
    return px, py


def fence_xy(team_code: str, offset: float = 0.0, n: int = 361):
    """Approximate an outfield fence curve in (x, y) for a team/park."""
    from mlb_vis_config import PARKS
    k = str(team_code).upper() if team_code else ""
    p = PARKS[k]
    ang_deg = np.array([-45, -20, 0, 20, 45], dtype=float)
    dist = np.array([p["LF"], p["LCF"], p["CF"], p["RCF"], p["RF"]], dtype=float) - float(offset)
    dist = np.clip(dist, 10.0, None)
    thetas = np.linspace(-45, 45, n)
    rs = np.interp(thetas, ang_deg, dist)
    rad = np.deg2rad(thetas)
    x = rs * np.sin(rad)
    y = rs * np.cos(rad)
    return x, y


def circle_xy(cx: float, cy: float, r: float, n: int = 90):
    """Parametric circle points centered at (cx, cy) with radius r."""
    th = np.linspace(0, 2 * np.pi, n)
    return cx + r * np.cos(th), cy + r * np.sin(th)


def rotated_rect_xy(cx: float, cy: float, w: float, h: float, angle_deg: float, n_close=True):
    """Axis-aligned rectangle centered at (cx, cy) then rotated by angle_deg."""
    ax = np.array([-w / 2, w / 2, w / 2, -w / 2], dtype=float)
    ay = np.array([-h / 2, -h / 2, h / 2, h / 2], dtype=float)
    ang = np.deg2rad(angle_deg)
    rx = ax * np.cos(ang) - ay * np.sin(ang) + cx
    ry = ax * np.sin(ang) + ay * np.cos(ang) + cy
    if n_close:
        rx = np.r_[rx, rx[0]]
        ry = np.r_[ry, ry[0]]
    return rx, ry


def diamond_square_xy(cx: float, cy: float, side: float):
    """Diamond - square rotated 45° path centered at (cx, cy)."""
    return rotated_rect_xy(cx, cy, side, side, angle_deg=45.0, n_close=True)


def home_plate_xy():
    """Returns a 5-sided home plate."""
    top_w = 17 / 12
    top_y = 1.00
    mid_y = 0.42
    pts = np.array(
        [
            [-top_w / 2, top_y],
            [top_w / 2, top_y],
            [top_w / 2, mid_y],
            [0.0, 0.0],
            [-top_w / 2, mid_y],
        ]
    )
    x = np.r_[pts[:, 0], pts[0, 0]]
    y = np.r_[pts[:, 1], pts[0, 1]]
    return x, y


def warning_track_ring(team_code: str, width: float = 15.0):
    """Builds a warning track by making a ring polygon between the fence and an inner curve."""
    xo, yo = fence_xy(team_code, offset=0.0)
    xi, yi = fence_xy(team_code, offset=width)
    xr = np.r_[xo, xi[::-1], xo[0]]
    yr = np.r_[yo, yi[::-1], yo[0]]
    return xr, yr


def foul_poles(team_code: str, height: float = 25):
    """Return the two foul poles at fence endpoints."""
    xo, yo = fence_xy(team_code, offset=0.0)
    lf_x, lf_y = xo[0], yo[0]
    rf_x, rf_y = xo[-1], yo[-1]
    return (lf_x, lf_y, lf_y + height), (rf_x, rf_y, rf_y + height)


def _build_field_layers(fig: go.Figure, team_code: str):
    """Draws: grass, warning track, infield dirt ring, inner infield grass,
      fence line, foul lines, foul poles, mound + rubber, bases, home plate - adds to Plotly."""
    fence_x, fence_y = fence_xy(team_code, 0.0)
    track_x, track_y = warning_track_ring(team_code, width=15.0)
    # Outfield grass
    out_x = [0] + list(fence_x) + [0]
    out_y = [0] + list(fence_y) + [0]

    # Infield dirt ring
    BASELINE_SIDE = 90.0
    BASEPATH_WIDTH_FT = 6.0
    d_outer = diamond_points(BASELINE_SIDE + BASEPATH_WIDTH_FT)
    d_inner = diamond_points(BASELINE_SIDE - BASEPATH_WIDTH_FT)
    do_x = [*d_outer[:, 0], d_outer[0, 0]]
    do_y = [*d_outer[:, 1], d_outer[0, 1]]
    di_x = [*d_inner[:, 0], d_inner[0, 0]]
    di_y = [*d_inner[:, 1], d_inner[0, 1]]
    infield_dirt_ring_x = np.r_[do_x, di_x[::-1], do_x[0]]
    infield_dirt_ring_y = np.r_[do_y, di_y[::-1], do_y[0]]

    # Bases, home plate, mound, rubber
    diamond_center = diamond_points(BASELINE_SIDE)
    first_x, first_y = diamond_center[1]
    second_x, second_y = diamond_center[2]
    third_x, third_y = diamond_center[3]
    base_side = 5
    b1x, b1y = diamond_square_xy(first_x, first_y, base_side)
    b2x, b2y = diamond_square_xy(second_x, second_y, base_side)
    b3x, b3y = diamond_square_xy(third_x, third_y, base_side)
    hp_x, hp_y = home_plate_xy()

    mound_cy, mound_r = 60.5, 9.0
    mx, my = circle_xy(0.0, mound_cy, mound_r, n=100)
    rub_w, rub_h = 4.0, 1.0
    rx, ry = rotated_rect_xy(0.0, mound_cy, rub_w, rub_h, angle_deg=0.0)

    (lfx, lfy0, lfy1), (rfx, rfy0, rfy1) = foul_poles(team_code, height=25)

    # Grass fill
    fig.add_trace(
        go.Scatter(
            x=out_x,
            y=out_y,
            mode="lines",
            line=dict(width=0, color="rgba(0,0,0,0)"),
            fill="toself",
            fillcolor="#0B6E4F",
            hoverinfo="skip",
            showlegend=False,
        )
    )

    # Warning track fill
    fig.add_trace(
        go.Scatter(
            x=track_x,
            y=track_y,
            mode="lines",
            line=dict(width=0, color="rgba(0,0,0,0)"),
            fill="toself",
            fillcolor="#C6946A",
            hoverinfo="skip",
            showlegend=False,
        )
    )

    # Infield dirt
    fig.add_trace(
        go.Scatter(
            x=infield_dirt_ring_x,
            y=infield_dirt_ring_y,
            mode="lines",
            line=dict(width=1, color="#775E45"),
            fill="toself",
            fillcolor="#9B7653",
            hoverinfo="skip",
            showlegend=False,
        )
    )

    # Inner infield grass
    fig.add_trace(
        go.Scatter(
            x=di_x,
            y=di_y,
            mode="lines",
            line=dict(width=0, color="rgba(0,0,0,0)"),
            fill="toself",
            fillcolor="#0B6E4F",
            hoverinfo="skip",
            showlegend=False,
        )
    )

    # Fence line
    fig.add_trace(
        go.Scatter(
            x=fence_x,
            y=fence_y,
            mode="lines",
            line=dict(width=2, color="#000"),
            hoverinfo="skip",
            showlegend=False,
        )
    )

    # Foul lines
    fig.add_trace(
        go.Scatter(
            x=[0, fence_x[0]],
            y=[0, fence_y[0]],
            mode="lines",
            line=dict(width=1, color="#DDD"),
            hoverinfo="skip",
            showlegend=False,
        )
    )
    fig.add_trace(
        go.Scatter(
            x=[0, fence_x[-1]],
            y=[0, fence_y[-1]],
            mode="lines",
            line=dict(width=1, color="#DDD"),
            hoverinfo="skip",
            showlegend=False,
        )
    )

    # Foul poles
    fig.add_trace(
        go.Scatter(
            x=[lfx, lfx],
            y=[lfy0, lfy1],
            mode="lines",
            line=dict(width=4, color="#FFD400"),
            hoverinfo="skip",
            showlegend=False,
        )
    )
    fig.add_trace(
        go.Scatter(
            x=[rfx, rfx],
            y=[rfy0, rfy1],
            mode="lines",
            line=dict(width=4, color="#FFD400"),
            hoverinfo="skip",
            showlegend=False,
        )
    )

    # Mound + rubber
    fig.add_trace(
        go.Scatter(
            x=mx,
            y=my,
            mode="lines",
            line=dict(width=1, color="#775E45"),
            fill="toself",
            fillcolor="#A88364",
            hoverinfo="skip",
            showlegend=False,
        )
    )
    fig.add_trace(
        go.Scatter(
            x=rx,
            y=ry,
            mode="lines",
            line=dict(width=1, color="#222"),
            fill="toself",
            fillcolor="#FFFFFF",
            hoverinfo="skip",
            showlegend=False,
        )
    )

    # Bases + home plate
    for bx, by in [(b1x, b1y), (b2x, b2y), (b3x, b3y)]:
        fig.add_trace(
            go.Scatter(
                x=bx,
                y=by,
                mode="lines",
                line=dict(width=1, color="#222"),
                fill="toself",
                fillcolor="#FFFFFF",
                hoverinfo="skip",
                showlegend=False,
            )
        )
    fig.add_trace(
        go.Scatter(
            x=hp_x,
            y=hp_y,
            mode="lines",
            line=dict(width=1, color="#222"),
            fill="toself",
            fillcolor="#FFFFFF",
            hoverinfo="skip",
            showlegend=False,
        )
    )

    return fence_x, fence_y


def _finish_layout(fig: go.Figure, fence_x, fence_y, player, year, team_code):
    """Finalize layout: axes, header, distance labels, and watermark logo."""
    pad = 35
    ymax = float(np.nanmax(fence_y)) + pad
    xmin, xmax = float(np.nanmin(fence_x)) - pad, float(np.nanmax(fence_x)) + pad

    fig.update_xaxes(visible=False, range=[xmin, xmax], scaleanchor="y", scaleratio=1, domain=[0.0, 0.68])
    fig.update_yaxes(visible=False, range=[-10, ymax])
    fig.update_layout(plot_bgcolor="#000000", paper_bgcolor="#000000", margin=dict(l=10, r=10, t=50, b=10))

    # Annotate fence distances on page
    ann = []
    thetas = np.linspace(-45, 45, 361)
    rs_all = np.hypot(fence_x, fence_y)
    for ang_deg in (-45, -20, 0, 20, 45):
        rs = np.interp(ang_deg, thetas, rs_all)
        rad = np.deg2rad(ang_deg)
        rlab = rs - 7
        ann.append(
            dict(
                x=rlab * np.sin(rad),
                y=rlab * np.cos(rad),
                xref="x",
                yref="y",
                text=f"{int(round(rs))}′",
                showarrow=False,
                font=dict(size=11, color="#DDD"),
                bgcolor="rgba(0,0,0,0.5)",
                bordercolor="rgba(255,255,255,0.25)",
                borderwidth=1,
            )
        )
    header = f"{player} — {year} Home Runs · {team_code}"
    ann.append(
        dict(
            x=0.01,
            y=1.10,
            xref="paper",
            yref="paper",
            text=header,
            showarrow=False,
            font=dict(size=18, color="#FFF"),
            xanchor="left",
            yanchor="top",
            bgcolor="rgba(0,0,0,0.6)",
            bordercolor="rgba(255,255,255,0.2)",
            borderwidth=1,
            borderpad=4,
        )
    )

    # Semi-transparent team logo watermark placed in Center Field
    images = []
    lurl = logo_url(team_code)
    if lurl:
        cf_dist = float(np.max(np.hypot(fence_x, fence_y)))
        logo_center_y = cf_dist * 0.55
        logo_size = min(120.0, cf_dist * 0.35)
        images = [
            dict(
                source=lurl,
                xref="x",
                yref="y",
                x=-logo_size / 2,
                y=logo_center_y + logo_size / 2,
                sizex=logo_size,
                sizey=logo_size,
                xanchor="left",
                yanchor="top",
                opacity=0.32,
                layer="above",
            )
        ]
    fig.update_layout(annotations=tuple(ann), images=images)


def _prep_hr_df(player: str, year: int):
    """Fetch and prepare the player's HR events and raw Statcast data.
  Steps:
  - Resolve player to MLBAM id
  - Pull Statcast batter events for the season window
  - Identify opponent team per event
  - Filter to home runs with valid spray coords
  - Convert hc_x/y to on-field feet (x, y)
  - Optionally reverse-lookup pitcher names
  - Build a minimal display table with order and counts"""
    pid = lookup_mlbam_id(player)
    start, end = season_dates(year)
    raw = statcast_batter(start, end, pid)

    # Find pitcher
    raw["inning_topbot"] = raw["inning_topbot"].astype(str).str.title()
    raw["opponent"] = np.where(raw["inning_topbot"] == "Bot", raw["away_team"], raw["home_team"])

    # Find Home runs with spray coordinates
    hr = raw[(raw["events"] == "home_run") & raw["hc_x"].notna() & raw["hc_y"].notna()].copy()
    if hr.empty:
        raise ValueError(f"No home runs with spray data for {player} in {year}.")
    hr["game_date"] = pd.to_datetime(hr["game_date"], errors="coerce")
    hr = hr.dropna(subset=["game_date"])
    hr["x"], hr["y"] = statcast_xy_to_feet(hr["hc_x"], hr["hc_y"])

    # Reverse-lookup of pitcher names via MLBAM ids
    if playerid_reverse_lookup is not None and "pitcher" in hr.columns:
        try:
            ids = sorted(set(int(x) for x in hr["pitcher"].dropna().astype(int).tolist()))
            if ids:
                pid_df = playerid_reverse_lookup(ids, key_type="mlbam")
                if not pid_df.empty:
                    pid_df["pitcher_name"] = (
                        pid_df["name_first"].str.title().fillna("")
                        + " "
                        + pid_df["name_last"].str.title().fillna("")
                    ).str.strip()
                    id2name = dict(zip(pid_df["key_mlbam"].astype(int), pid_df["pitcher_name"]))
                    hr["pitcher_name"] = hr["pitcher"].astype(int).map(id2name)
        except Exception:
            hr["pitcher_name"] = hr.get("pitcher", "").astype(str)
    if "pitcher_name" not in hr.columns:
        hr["pitcher_name"] = hr.get("pitcher", "").astype(str)

    # Pitch type
    if "pitch_name" in hr.columns:
        hr["pitch_display"] = hr["pitch_name"].fillna(hr.get("pitch_type", ""))
    else:
        hr["pitch_display"] = hr.get("pitch_type", "")

    # Order events chronologically
    for c in ["at_bat_number", "pitch_number", "game_pk"]:
        if c not in hr.columns:
            hr[c] = 0
    hr = hr.sort_values(["game_date", "game_pk", "at_bat_number", "pitch_number"]).reset_index(drop=True)
    hr["HR#"] = np.arange(1, len(hr) + 1)
    return hr, raw


def build_animation(player: str, year: int):
    """ Build an animated HR spray chart for a player/season using Plotly."""
    # --- data ---
    hr, raw = _prep_hr_df(player, year)
    home_team = pick_primary_home_team(raw)
    team_code = str(home_team).upper() if home_team else ""
    tcolor = team_color(team_code)

    # --- figure + field layers ---
    fig = go.Figure()
    fence_x, fence_y = _build_field_layers(fig, team_code)

    # Trail of completed HRs
    trail_idx = len(fig.data)
    fig.add_trace(
        go.Scatter(
            x=[],
            y=[],
            mode="markers",
            marker=dict(size=10, opacity=0.35, color=tcolor),
            hoverinfo="skip",
            showlegend=False,
        )
    )

    # Path of current ball
    path_idx = len(fig.data)
    fig.add_trace(
        go.Scatter(
            x=[],
            y=[],
            mode="lines",
            line=dict(width=1, dash="dot", color=tcolor),
            hoverinfo="skip",
            showlegend=False,
        )
    )

    # Current ball marker
    ball_idx = len(fig.data)
    fig.add_trace(
        go.Scatter(
            x=[],
            y=[],
            mode="markers",
            marker=dict(size=10, color=tcolor, line=dict(width=1, color="#000")),
            name="Ball",
        )
    )

    # Table
    table_domain = dict(x=[0.57, 0.94], y=[0.10, 0.90])
    table_idx = len(fig.data)
    fig.add_trace(
        go.Table(
            domain=table_domain,
            header=dict(
                values=["Date", "Pitcher", "LA (°)", "EV (mph)", "Velo (mph)", "Pitch"],
                fill_color="#222",
                font=dict(color="#fff", size=11),
                align="left",
            ),
            cells=dict(
                values=[[], [], [], [], [], []],
                fill_color="#111",
                font=dict(color="#eee", size=11),
                align="left",
                height=20,
            ),
            columnwidth=[0.22, 0.28, 0.12, 0.14, 0.14, 0.20],
        )
    )

    _finish_layout(fig, fence_x, fence_y, player, year, team_code)

    # --- frames ---
    frames = []
    steps_per_hr = 20
    path_curve = 0.08

    for _, row in hr.iterrows():
        hrn = int(row["HR#"])
        prev = hr.loc[hr["HR#"] < hrn, ["x", "y"]]
        path_x, path_y = ball_flight_path(float(row["x"]), float(row["y"]), steps=steps_per_hr, curve=path_curve)

        date_str = pd.to_datetime(row["game_date"]).strftime("%Y-%m-%d")
        pitcher = row.get("pitcher_name") if pd.notna(row.get("pitcher_name")) else row.get("pitcher")
        la_str = f"{row.get('launch_angle', np.nan):.0f}" if pd.notna(row.get("launch_angle")) else ""
        ev_str = f"{row.get('launch_speed', np.nan):.0f}" if pd.notna(row.get("launch_speed")) else ""
        pv_str = f"{row.get('release_speed', np.nan):.0f}" if pd.notna(row.get("release_speed")) else ""
        ptype = str(row.get("pitch_display", ""))

        for s in range(steps_per_hr):
            is_final = s == steps_per_hr - 1
            label = (
                f"HR #{hrn} — {date_str} vs {row['opponent']} | "
                f"EV {row.get('launch_speed', np.nan):.0f} mph · "
                f"LA {row.get('launch_angle', np.nan):.0f}°"
            )

            frames.append(
                go.Frame(
                    name=f"{hrn}_{s}",
                    data=[
                        go.Scatter(
                            x=prev["x"], y=prev["y"], mode="markers", marker=dict(size=10, opacity=0.35, color=tcolor)
                        ),
                        go.Scatter(
                            x=path_x[: s + 1], y=path_y[: s + 1], mode="lines", line=dict(width=1, dash="dot", color=tcolor)
                        ),
                        go.Scatter(
                            x=[path_x[s]],
                            y=[path_y[s]],
                            mode="markers",
                            marker=dict(size=12 if is_final else 10, color=tcolor, line=dict(width=1, color="#000")),
                            text=[label] if is_final else [""],
                            hovertemplate="%{text}<extra></extra>",
                        ),
                        go.Table(
                            domain=table_domain,
                            header=dict(
                                values=["Date", "Pitcher", "LA (°)", "EV (mph)", "Velo (mph)", "Pitch"],
                                fill_color="#222",
                                font=dict(color="#fff", size=11),
                                align="left",
                            ),
                            cells=dict(
                                values=[[date_str], [pitcher], [la_str], [ev_str], [pv_str], [ptype]],
                                fill_color="#111",
                                font=dict(color="#eee", size=11),
                                align="left",
                                height=20,
                            ),
                            columnwidth=[0.22, 0.28, 0.12, 0.14, 0.14, 0.20],
                        ),
                    ],
                    traces=[trail_idx, path_idx, ball_idx, table_idx],
                )
            )

    fig.frames = frames

    last_frame_name = f"{int(hr['HR#'].iloc[-1])}_{steps_per_hr-1}"
    fig.update_layout(
        showlegend=False,
        updatemenus=[
            {
                "type": "buttons",
                "x": 1.01,
                "y": 1.02,
                "direction": "right",
                "buttons": [
                    {
                        "label": "Play",
                        "method": "animate",
                        "args": [None, {"fromcurrent": True, "frame": {"duration": 100, "redraw": True}, "transition": {"duration": 0}}],
                    },
                    {
                        "label": "Pause",
                        "method": "animate",
                        "args": [[None], {"mode": "immediate", "frame": {"duration": 0, "redraw": False}, "transition": {"duration": 0}}],
                    },
                ],
            },
            {
                "type": "buttons",
                "x": 1.005,
                "y": 0.92,
                "direction": "right",
                "buttons": [
                    {
                        "label": "Skip to End",
                        "method": "animate",
                        "args": [
                            [last_frame_name],
                            {"mode": "immediate", "frame": {"duration": 0, "redraw": True}, "transition": {"duration": 0}},
                        ],
                    }
                ],
            },
        ],
        sliders=[
            {
                "active": 0,
                "pad": {"t": 20},
                "x": 0.06,
                "xanchor": "left",
                "len": 0.86,
                "currentvalue": {"prefix": "", "visible": False},
                "font": {"color": "white"},
                "steps": [
                    {
                        "label": f"HR {int(r['HR#'])}",
                        "method": "animate",
                        "args": [
                            [f"{int(r['HR#'])}_{steps_per_hr-1}"],
                            {"frame": {"duration": 0, "redraw": True}, "transition": {"duration": 0}},
                        ],
                    }
                    for _, r in hr.iterrows()
                ],
            }
        ],
    )
    return fig


def build_interactive_click(player: str, year: int):
    """Build a clickable HR spray chart (FigureWidget). Clicking a dot updates the table."""
    from plotly.graph_objs import FigureWidget

    hr, raw = _prep_hr_df(player, year)
    home_team = pick_primary_home_team(raw)
    team_code = str(home_team).upper() if home_team else ""
    tcolor = team_color(team_code)

    fig = FigureWidget()
    fence_x, fence_y = _build_field_layers(fig, team_code)

    table_domain = dict(x=[0.57, 0.94], y=[0.10, 0.90])
    table_idx = len(fig.data)
    fig.add_trace(
        go.Table(
            domain=table_domain,
            header=dict(
                values=["Date", "Pitcher", "LA (°)", "EV (mph)", "Velo (mph)", "Pitch"],
                fill_color="#222",
                font=dict(color="#fff", size=11),
                align="left",
            ),
            cells=dict(
                values=[[], [], [], [], [], []],
                fill_color="#111",
                font=dict(color="#eee", size=11),
                align="left",
                height=20,
            ),
            columnwidth=[0.22, 0.28, 0.12, 0.14, 0.14, 0.20],
        )
    )

    date_strs = hr["game_date"].dt.strftime("%Y-%m-%d").tolist()

    if "pitcher_name" in hr.columns:
        pitchers = hr["pitcher_name"].where(hr["pitcher_name"].notna(), hr.get("pitcher", "").astype(str)).tolist()
    else:
        pitchers = hr.get("pitcher", "").astype(str).tolist()

    la_vals = hr["launch_angle"] if "launch_angle" in hr.columns else pd.Series([np.nan] * len(hr))
    ev_vals = hr["launch_speed"] if "launch_speed" in hr.columns else pd.Series([np.nan] * len(hr))
    velo_vals = hr["release_speed"] if "release_speed" in hr.columns else pd.Series([np.nan] * len(hr))

    la_strs = [f"{v:.0f}" if pd.notna(v) else "" for v in la_vals]
    ev_strs = [f"{v:.0f}" if pd.notna(v) else "" for v in ev_vals]
    velo_strs = [f"{v:.0f}" if pd.notna(v) else "" for v in velo_vals]
    pitch_names = (hr["pitch_display"].astype(str) if "pitch_display" in hr.columns else pd.Series([""] * len(hr))).tolist()

    fig.add_trace(
        go.Scatter(
            x=hr["x"],
            y=hr["y"],
            mode="markers",
            marker=dict(size=10, color=tcolor, line=dict(width=1, color="#000")),
            selected=dict(marker=dict(size=16, color=tcolor)),
            unselected=dict(marker=dict(opacity=0.35)),
            customdata=np.stack([date_strs, pitchers, la_strs, ev_strs, velo_strs, pitch_names], axis=-1),
            text=[f"#{n}" for n in hr["HR#"]],
            hovertemplate="HR %{text}<extra></extra>",
            showlegend=False,
        )
    )
    dots_idx = len(fig.data) - 1

    _finish_layout(fig, fence_x, fence_y, player, year, team_code)

    def _set_table_from_index(i: int):
        cd = fig.data[dots_idx].customdata[i]
        fig.data[table_idx].cells.values = [[cd[0]], [cd[1]], [cd[2]], [cd[3]], [cd[4]], [cd[5]]]
        fig.data[dots_idx].selectedpoints = [i]

    _set_table_from_index(len(hr) - 1)

    def _on_click(trace, points, selector):
        if points.point_inds:
            _set_table_from_index(points.point_inds[0])

    fig.data[dots_idx].on_click(_on_click)
    return fig


Overwriting mlb_vis_core.py


In [8]:
# @title Building Animation & Clickable Viz

import plotly.io as pio
from importlib import reload

# Colab renderer + widget callbacks for FigureWidget
pio.renderers.default = "colab"
try:
    from google.colab import output
    output.enable_custom_widget_manager()
except Exception:
    pass

# Reload local modules written by the earlier %%writefile cells
import mlb_vis_config
import mlb_vis_core as vis
reload(mlb_vis_config)
reload(vis)

# Animated version with Play/Pause + Skip to End
fig = vis.build_animation(PLAYER, YEAR)
fig.show()

# Clickable version: click a dot to update the right-hand table
fig_click = vis.build_interactive_click(PLAYER, YEAR)
fig_click


Gathering player lookup table. This may take a moment.
Gathering Player Data


Gathering Player Data


FigureWidget({
    'data': [{'fill': 'toself',
              'fillcolor': '#0B6E4F',
              'hoverinfo': 'skip',
              'line': {'color': 'rgba(0,0,0,0)', 'width': 0},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter',
              'uid': 'cd0ab735-6661-43de-bd2b-f26258522271',
              'x': [0, -245.36605307173198, -244.5747152760034, ...,
                    246.5459565044801, 247.48737341529161, 0],
              'y': [0, 245.366053071732, 246.71839949152934, ...,
                    248.70691854327455, 247.48737341529164, 0]},
             {'fill': 'toself',
              'fillcolor': '#C6946A',
              'hoverinfo': 'skip',
              'line': {'color': 'rgba(0,0,0,0)', 'width': 0},
              'mode': 'lines',
              'showlegend': False,
              'type': 'scatter',
              'uid': '89acfeb8-a495-4334-b8d1-bc886a6bffe2',
              'x': array([-245.36605307, -244.57471528, -243.7762421